In [1]:
from pyspark.sql import SparkSession
import opmDataset
from pandas import *
from mmtfPyspark.io import mmtfReader
from ipywidgets import interact, IntSlider
import py3Dmol

#### Configure Spark

In [2]:
spark = SparkSession.builder.master("local[4]").appName("OPM_DEMO").getOrCreate()
sc = spark.sparkContext

In [3]:
dataset = opmDataset.get_dataset()

In [4]:
dataset.toPandas().head()

,Family,ProteinName,PDBID,Species,Localization,Num.Subunits,Num.TMSec.Structs.,HydrophobicThicknessorDepth(Å),TiltAngle(°),ΔGtransfer(kcal/mol)
0,1.1.01.01.,Cruxrhodopsin-3,4JR8,Haloarcula vallismortis,Archaebac.,3,21,31.8 ± 1.2,0 ± 0,-130.4
1,1.1.01.01.,"Bacteriorhodopsin, K state, with lipids",1IW6,Halobacterium salinarum,Archaebac.,3,21,30.0 ± 0.6,0 ± 0,-112.6
2,1.1.01.01.,"Bacteriorhodopsin, trimer, complex with annula...",2ZZL,Halobacterium salinarum,Archaebac.,3,21,29.8 ± 0.6,0 ± 0,-119.0
3,1.1.01.01.,Bacteriorhodopsin-I,4PXK,Haloarcula marismortui,Archaebac.,1,7,31.8 ± 1.4,11 ± 1,-72.7
4,1.1.01.01.,Bacteriorhodopsin,5AZD,Thermus thermophilus,Bact. Gram-neg inner,4,28,30.0 ± 0.5,3 ± 1,-121.3


In [10]:
pdbids = dataset.toPandas()['PDBID'].tolist()
#print(pdbids)
structures = mmtfReader.download_full_mmtf_files(pdbids, sc)
type(structures)

pyspark.rdd.PipelinedRDD

In [13]:
print(structures.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 15, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ctlee/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/ctlee/anaconda3/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/ctlee/anaconda3/lib/python3.6/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/ctlee/anaconda3/lib/python3.6/site-packages/pyspark/rdd.py", line 2438, in pipeline_func
    return func(split, prev_

In [ ]:
def view_proteins(pdb_ids, chain_ids, groups, var_ids, ligand_id, distance=4.5):
    
    def view3d(i=0):
        
        print(f"PDB: {pdb_ids[i]}, chain: {chain_ids[i]}, group: {groups[i]}, variation: {var_ids[i]}")

        mutation = {'resi': groups[i], 'chain': chain_ids[i]}
        neighbors = {'resi': groups[i], 'chain': chain_ids[i], 'byres': 'true', 'expand': distance}

        viewer = py3Dmol.view(query='pdb:' + pdb_ids[i]) 
        viewer.setStyle(neighbors, {'stick': {'colorscheme': 'orangeCarbon', 'radius': 0.1}});
        viewer.setStyle(mutation, {'stick': {'colorscheme': 'redCarbon'}})
        viewer.addResLabels(mutation)
        viewer.setStyle({'resn': ligand_id}, {'sphere': {'colorscheme': 'greenCarbon'}})
        viewer.zoomTo(neighbors)

        return viewer.show()

    s_widget = IntSlider(min=0, max=len(pdb_ids)-1, description='Structure', continuous_update=False)
    return interact(view3d, i=s_widget)

In [ ]:
spark.stop()